In [231]:
!pip install mysql-connector-python

In [232]:
import mysql.connector
from mysql.connector.constants import ClientFlag

config = {
    'user': 'lel',
    'password': 'not_a_password',
    'host': '192.168.1.1',
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'ssl/server-ca.pem',
    'ssl_cert': 'ssl/client-cert.pem',
    'ssl_key': 'ssl/client-key.pem'
}

# now we establish our connection
cnxn = mysql.connector.connect(**config)

In [233]:
config['database'] = 'ytdatabase'  #Add new database to config dict
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

In [234]:
import pandas as pd

In [235]:
df = pd.read_csv('Space_Corrected.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,0,0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,1,1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,2,2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,3,3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,4,4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [236]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Company Name', 'Location', 'Datum',
       'Detail', 'Status Rocket', ' Rocket', 'Status Mission'],
      dtype='object')

In [237]:
#The heck are those two unnamed columns? Gotcha drop em all!
col_names_list = ['Company Name', 'Location', 'Datum', 'Detail', 'Status Rocket', ' Rocket', 'Status Mission']
df = pd.read_csv('Space_Corrected.csv', usecols = col_names_list)
df.head()

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


In [238]:
df.dtypes

Company Name      object
Location          object
Datum             object
Detail            object
Status Rocket     object
 Rocket           object
Status Mission    object
dtype: object

In [239]:
"""cursor.execute("CREATE TABLE space_missions_cleaned ("
               "company_name VARCHAR(255),"
               "location VARCHAR(255),"
               "datum DATE,"
               "detail VARCHAR(255),"
               "status_rocket VARCHAR(255),"
               "rocket DECIMAL,"
               "status_mission VARCHAR(255) )")

cnxn.commit()"""

'cursor.execute("CREATE TABLE space_missions_cleaned ("\n               "company_name VARCHAR(255),"\n               "location VARCHAR(255),"\n               "datum DATE,"\n               "detail VARCHAR(255),"\n               "status_rocket VARCHAR(255),"\n               "rocket DECIMAL,"\n               "status_mission VARCHAR(255) )")\n\ncnxn.commit()'

In [240]:
df.isnull().sum()

Company Name         0
Location             0
Datum                0
Detail               0
Status Rocket        0
 Rocket           3360
Status Mission       0
dtype: int64

In [241]:
df[' Rocket'] = df[' Rocket'].fillna(0)

In [242]:
df.isnull().sum()

Company Name      0
Location          0
Datum             0
Detail            0
Status Rocket     0
 Rocket           0
Status Mission    0
dtype: int64

In [243]:
df = df.rename(columns={' Rocket': 'Rocket'})

In [244]:
df.columns

Index(['Company Name', 'Location', 'Datum', 'Detail', 'Status Rocket',
       'Rocket', 'Status Mission'],
      dtype='object')

In [245]:
list(df.itertuples(index=False, name = None))[1]

('CASC',
 'Site 9401 (SLS-2), Jiuquan Satellite Launch Center, China',
 'Thu Aug 06, 2020 04:01 UTC',
 'Long March 2D | Gaofen-9 04 & Q-SAT',
 'StatusActive',
 '29.75 ',
 'Success')

In [246]:
convert_dict = {'Company Name': str,
                'Location': str,
                 'Datum' : str,
                'Detail': str,
                'Status Rocket': str,
                'Rocket': float,
                'Status Mission': str
               }
  
df = df.astype(convert_dict)
df.dtypes

ValueError: ignored

In [247]:
#Lets find those :P
df[df['Rocket'] == '5,000.0 ']

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
1916,RVSN USSR,"Site 110/37, Baikonur Cosmodrome, Kazakhstan","Tue Nov 15, 1988 03:00 UTC",Energiya/Buran | Buran,StatusRetired,"5,000.0",Success
2000,RVSN USSR,"Site 250, Baikonur Cosmodrome, Kazakhstan","Fri May 15, 1987 17:30 UTC",Energiya/Polyus | Polyus Space Station,StatusRetired,"5,000.0",Success


In [248]:
df["Rocket"].replace({"5,000.0 ": 5000}, inplace = True)

In [249]:
df[df['Rocket'] == '5,000.0 ']

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission


In [250]:
df = df.astype(convert_dict)
df.dtypes

ValueError: ignored

In [251]:
df["Rocket"].replace({'1,160.0 ': 1160}, inplace = True)
df = df.astype(convert_dict)
df.dtypes

Company Name       object
Location           object
Datum              object
Detail             object
Status Rocket      object
Rocket            float64
Status Mission     object
dtype: object

In [252]:
df_copy = df.copy()

In [253]:
#Let's extract the date from the 'Datum' column

In [254]:
for date in list(df['Datum'].unique()):
  print(date)

Fri Aug 07, 2020 05:12 UTC
Thu Aug 06, 2020 04:01 UTC
Tue Aug 04, 2020 23:57 UTC
Thu Jul 30, 2020 21:25 UTC
Thu Jul 30, 2020 11:50 UTC
Sat Jul 25, 2020 03:13 UTC
Thu Jul 23, 2020 14:26 UTC
Thu Jul 23, 2020 04:41 UTC
Mon Jul 20, 2020 21:30 UTC
Sun Jul 19, 2020 21:58 UTC
Wed Jul 15, 2020 13:46 UTC
Fri Jul 10, 2020 04:17 UTC
Thu Jul 09, 2020 12:11 UTC
Mon Jul 06, 2020 01:00 UTC
Sat Jul 04, 2020 23:44 UTC
Sat Jul 04, 2020 21:19 UTC
Fri Jul 03, 2020 03:10 UTC
Tue Jun 30, 2020 20:10 UTC
Tue Jun 23, 2020 01:43 UTC
Wed Jun 17, 2020 07:19 UTC
Sat Jun 13, 2020 09:21 UTC
Sat Jun 13, 2020 05:12 UTC
Wed Jun 10, 2020 18:31 UTC
Thu Jun 04, 2020 01:25 UTC
Sun May 31, 2020 08:53 UTC
Sat May 30, 2020 19:22 UTC
Fri May 29, 2020 20:13 UTC
Mon May 25, 2020 19:50 UTC
Fri May 22, 2020 07:31 UTC
Wed May 20, 2020 17:31 UTC
Sun May 17, 2020 13:14 UTC
Tue May 12, 2020 01:16 UTC
Tue May 05, 2020 10:00 UTC
Sat Apr 25, 2020 01:51 UTC
Wed Apr 22, 2020 19:30 UTC
Wed Apr 22, 2020 03:59 UTC
Thu Apr 09, 2020 11:46 UTC
T

In [255]:
#We see that we can extract part of the string (for example Sep 08, 1967) and then for each month convert it to number
#In SQL date should be in format YYYY-MM-DD
date_str_test = 'Wed Mar 26, 1969 12:30 UTC'

print(date_str_test[4:16])

Mar 26, 1969


In [256]:
date_str_test[4:16].replace(',', '')

'Mar 26 1969'

In [257]:
date_str_test[4:16].replace(',', '').split(" ")

['Mar', '26', '1969']

In [258]:
year = date_str_test[4:16].replace(',', '').split(" ")[2]
month = date_str_test[4:16].replace(',', '').split(" ")[0]
day = date_str_test[4:16].replace(',', '').split(" ")[1]
print(year+'-'+month+'-'+day)

1969-Mar-26


In [259]:
df['Datum'][0]

'Fri Aug 07, 2020 05:12 UTC'

In [260]:
prob_string = 'Thu Aug 29, 2019'
print(prob_string[4:16].replace(',', '').split(" "))

['Aug', '29', '2019']


In [261]:
df['Datum'][5]

'Sat Jul 25, 2020 03:13 UTC'

In [262]:
len(df['Datum'][5])

26

In [263]:
df['Datum'][5][0:4], df['Datum'][5][4:6], df['Datum'][5][6:8]

('Sat ', 'Ju', 'l ')

In [264]:
#There are three unique types of date strings in 'Datum' column
#1. Tue Jan 29, 1991
#2. 2020-07-20
#3. Fri Aug 07, 2020 05:12 UTC
len('Tue Jan 29, 1991'), len('2020-07-20')

(16, 10)

In [265]:
date_type_1 = 'Tue Jan 29, 1991'
date_type_1[4:16].replace(',', '').split(" ")

['Jan', '29', '1991']

In [266]:
lst = []

In [267]:
for date in df['Datum']:
    lst.append(len(date))

In [268]:
set(lst)

{16, 26}

In [334]:
df = df_copy.copy()
df.head()

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.00,Success


In [336]:
df.iat[0,2]


'Fri Aug 07, 2020 05:12 UTC'

In [339]:
for i in range(0,len(df['Datum'])):
    print(df.iat[i,2][0:16])

Fri Aug 07, 2020
Thu Aug 06, 2020
Tue Aug 04, 2020
Thu Jul 30, 2020
Thu Jul 30, 2020
Sat Jul 25, 2020
Thu Jul 23, 2020
Thu Jul 23, 2020
Mon Jul 20, 2020
Sun Jul 19, 2020
Wed Jul 15, 2020
Fri Jul 10, 2020
Thu Jul 09, 2020
Mon Jul 06, 2020
Sat Jul 04, 2020
Sat Jul 04, 2020
Fri Jul 03, 2020
Tue Jun 30, 2020
Tue Jun 23, 2020
Wed Jun 17, 2020
Sat Jun 13, 2020
Sat Jun 13, 2020
Wed Jun 10, 2020
Thu Jun 04, 2020
Sun May 31, 2020
Sat May 30, 2020
Fri May 29, 2020
Mon May 25, 2020
Fri May 22, 2020
Wed May 20, 2020
Sun May 17, 2020
Tue May 12, 2020
Tue May 05, 2020
Sat Apr 25, 2020
Wed Apr 22, 2020
Wed Apr 22, 2020
Thu Apr 09, 2020
Thu Apr 09, 2020
Thu Mar 26, 2020
Tue Mar 24, 2020
Sat Mar 21, 2020
Wed Mar 18, 2020
Mon Mar 16, 2020
Mon Mar 16, 2020
Mon Mar 09, 2020
Sat Mar 07, 2020
Thu Feb 20, 2020
Wed Feb 19, 2020
Tue Feb 18, 2020
Mon Feb 17, 2020
Sat Feb 15, 2020
Mon Feb 10, 2020
Sun Feb 09, 2020
Sun Feb 09, 2020
Thu Feb 06, 2020
Fri Jan 31, 2020
Wed Jan 29, 2020
Sun Jan 19, 2020
Thu Jan 16, 20

In [340]:
df.iat[0,2][0:16].replace(',', '').split(" ")

['Fri', 'Aug', '07', '2020']

In [341]:
for i in range(0,len(df['Datum'])):
      year = df.iat[i,2][0:16].replace(',', '').split(" ")[3]
      month = df.iat[i,2][0:16].replace(',', '').split(" ")[1]
      day = df.iat[i,2][0:16].replace(',', '').split(" ")[2]
      if month == 'Jan':
         df.iat[i,2] = year+'-01-'+day

      if month == 'Feb':
        df.iat[i,2] = year+'-02-'+day

      if month == 'Mar':
        df.iat[i,2] = year+'-03-'+day

      if month == 'Apr':
        df.iat[i,2] = year+'-04-'+day

      if month == 'May':
        df.iat[i,2] = year+'-05-'+day

      if month == 'Jun':
        df.iat[i,2] = year+'-06-'+day

      if month == 'Jul':
        df.iat[i,2] = year+'-07-'+day

      if month == 'Aug':
        df.iat[i,2] = year+'-08-'+day

      if month == 'Sep':
        df.iat[i,2] = year+'-09-'+day

      if month == 'Oct':
        df.iat[i,2] = year+'-10-'+day

      if month == 'Nov':
        df.iat[i,2] = year+'-11-'+day

      if month == 'Dec':
        df.iat[i,2] = year+'-12-'+day

In [343]:
df.head(10)

,Company Name,Location,Datum,Detail,Status Rocket,Rocket,Status Mission
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA",2020-08-07,Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.00,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...",2020-08-06,Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA",2020-08-04,Starship Prototype | 150 Meter Hop,StatusActive,0.00,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan",2020-07-30,Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.00,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA",2020-07-30,Atlas V 541 | Perseverance,StatusActive,145.00,Success
5,CASC,"LC-9, Taiyuan Satellite Launch Center, China",2020-07-25,"Long March 4B | Ziyuan-3 03, Apocalypse-10 & N...",StatusActive,64.68,Success
6,Roscosmos,"Site 31/6, Baikonur Cosmodrome, Kazakhstan",2020-07-23,Soyuz 2.1a | Progress MS-15,StatusActive,48.50,Success
7,CASC,"LC-101, Wenchang Satellite Launch Center, China",2020-07-23,Long March 5 | Tianwen-1,StatusActive,0.00,Success
8,SpaceX,"SLC-40, Cape Canaveral AFS, Florida, USA",2020-07-20,Falcon 9 Block 5 | ANASIS-II,StatusActive,50.00,Success
9,JAXA,"LA-Y1, Tanegashima Space Center, Japan",2020-07-19,H-IIA 202 | Hope Mars Mission,StatusActive,90.00,Success


In [344]:
df.dtypes

Company Name       object
Location           object
Datum              object
Detail             object
Status Rocket      object
Rocket            float64
Status Mission     object
dtype: object

In [346]:
df_copy = df.copy()

In [347]:
df['Datum'] = pd.to_datetime(df['Datum'])

In [348]:
df.dtypes

Company Name              object
Location                  object
Datum             datetime64[ns]
Detail                    object
Status Rocket             object
Rocket                   float64
Status Mission            object
dtype: object

In [355]:
cursor.execute("CREATE TABLE space_missions_cleaned ("
               "company_name VARCHAR(255),"
               "location VARCHAR(255),"
               "datum DATETIME,"
               "detail VARCHAR(255),"
               "status_rocket VARCHAR(255),"
               "rocket FLOAT(6,2),"
               "status_mission VARCHAR(255) )")

cnxn.commit()

In [356]:
query = ("INSERT INTO space_missions_cleaned (company_name, location, datum, detail, status_rocket, rocket, status_mission) "
         "VALUES (%s, %s, %s, %s, %s, %s, %s)")

cursor.executemany(query, list(df.itertuples(index=False, name = None)))
cnxn.commit()

In [357]:
cursor.execute('SELECT * FROM space_missions_cleaned LIMIT 10')

output = cursor.fetchall()
for row in output:
  print(row)

('SpaceX', 'LC-39A, Kennedy Space Center, Florida, USA', datetime.datetime(2020, 8, 7, 0, 0), 'Falcon 9 Block 5 | Starlink V1 L9 & BlackSky', 'StatusActive', 50.0, 'Success')
('CASC', 'Site 9401 (SLS-2), Jiuquan Satellite Launch Center, China', datetime.datetime(2020, 8, 6, 0, 0), 'Long March 2D | Gaofen-9 04 & Q-SAT', 'StatusActive', 29.75, 'Success')
('SpaceX', 'Pad A, Boca Chica, Texas, USA', datetime.datetime(2020, 8, 4, 0, 0), 'Starship Prototype | 150 Meter Hop', 'StatusActive', 0.0, 'Success')
('Roscosmos', 'Site 200/39, Baikonur Cosmodrome, Kazakhstan', datetime.datetime(2020, 7, 30, 0, 0), 'Proton-M/Briz-M | Ekspress-80 & Ekspress-103', 'StatusActive', 65.0, 'Success')
('ULA', 'SLC-41, Cape Canaveral AFS, Florida, USA', datetime.datetime(2020, 7, 30, 0, 0), 'Atlas V 541 | Perseverance', 'StatusActive', 145.0, 'Success')
('CASC', 'LC-9, Taiyuan Satellite Launch Center, China', datetime.datetime(2020, 7, 25, 0, 0), 'Long March 4B | Ziyuan-3 03, Apocalypse-10 & NJU-HKU 1', 'Statu

In [ ]:
#Now, that looks much cleaner!